## Load modules

In [1]:
import pandas as pd
import os

## Load all files
Get the %Area value from each of the 8 images per mouse. Then put all results in a new df and calculate the average in each mouse.

In [2]:
folders = [folder for folder in os.listdir(os.getcwd()) if len(folder) == 6]
paths = []

# Make list with paths to each folder
for folder in folders:
    path = os.path.join(os.getcwd() , folder)
    paths.append(path)

# Parse the vessel density data from each file
vessel_areas_dict = {'Mouse ID': [], 
                     '%Area image': []} 
for path in paths:
    for filename in os.listdir(path):
        if 'area' in filename:
            file = os.path.join(path, filename)
            sample_df = pd.read_csv(file, sep = '\t')
            vessel_areas_dict['Mouse ID'].append(filename[:6])
            vessel_areas_dict['%Area image'].append(sample_df['%Area'].values[0])
    
# Put all data into a dataframe and calculate the mean
vessel_areas_df = pd.DataFrame.from_dict(vessel_areas_dict)
vessel_areas_df.groupby('Mouse ID').describe()

%Area image                                                  \
               count       mean       std     min       25%      50%   
Mouse ID                                                               
445776           8.0   5.569000  1.497146   4.065   4.52800   5.0585   
445777           8.0   9.787750  1.282067   7.723   9.38725   9.5435   
445778           8.0   9.078000  1.700019   7.133   7.79850   9.0140   
445779           8.0   6.578625  0.973848   5.570   5.86800   6.3675   
445780           8.0  11.621750  1.384459   9.971  10.86675  11.5020   
445781           8.0   7.120375  1.014775   5.910   6.26825   6.9190   
445782           8.0   5.732000  0.979580   4.734   5.05975   5.6180   
446733           8.0   5.545125  0.829705   4.447   5.07725   5.4160   
446734           8.0   8.362375  1.258285   6.438   7.68725   8.5035   
446735           8.0   9.196125  1.618826   5.989   8.75225   9.4785   
452400           8.0   8.195250  1.430119   5.978   7.31450   8.2870   
452401           8.0   7.054500  0.890800   5.801   6.46050   6.9920   
452412           8.0  12.509625  1.151664  11.253  11.72500  12.1390   
452413           7.0   9.199714  1.486622   6.634   8.52450   9.8970   
452414           8.0  11.171625  1.560767   9.117   9.99000  11.1715   
452415           8.0  10.309125  1.307835   7.732   9.86575  10.2860   
466736           8.0   9.699500  1.286174   7.683   9.18675  10.0415   
468717           8.0   8.952250  1.226784   6.677   8.36575   9.0330   
468718           8.0   8.426875  2.298612   4.240   7.55250   8.4815   
468719           8.0  11.028500  1.120081   9.370  10.14825  11.3555   
468720           8.0  10.701500  1.883044   7.505   9.83550  10.4295   
468721           8.0  12.606375  1.813934   9.981  11.62050  12.5370   

                            
               75%     max  
Mouse ID                    
445776     6.55825   7.773  
445777    10.18250  12.319  
445778    10.26125  11.700  
445779     6.89500   8.346  
445780    12.12325  13.735  
445781     8.07625   8.493  
445782     5.97250   7.817  
446733     5.83700   7.039  
446734     9.34775   9.912  
446735    10.09400  11.234  
452400     8.95875  10.553  
452401     7.57175   8.530  
452412    13.06300  14.608  
452413     9.96600  10.886  
452414    11.84375  13.840  
452415    11.16050  11.948  
466736    10.56050  10.983  
468717     9.85050  10.560  
468718     9.91075  11.669  
468719    11.86800  12.316  
468720    11.99775  13.551  
468721    14.19000  14.876

## Add mouse infodata (genotype and surgery)

In [32]:
infodata = pd.read_csv('3WK All Features.csv')
infodata.rename(columns={'MOUSE': 'Mouse ID'}, inplace= True)

# Make sure the 'Mouse ID' column has the same dtype across dataframes
# and perfom a left join
vessel_areas_df['Mouse ID'] = vessel_areas_df['Mouse ID'].astype(infodata['Mouse ID'].dtype)
result = vessel_areas_df.merge(infodata[['Mouse ID','Genotype', 'Surgery']], on='Mouse ID', how = 'left')
display(result)

,Mouse ID,%Area image,Genotype,Surgery
0,445776,4.065,CKO,TAC
1,445776,4.066,CKO,TAC
2,445776,7.657,CKO,TAC
3,445776,5.416,CKO,TAC
4,445776,4.682,CKO,TAC
...,...,...,...,...
170,452412,12.836,Wt,Sham
171,452412,11.253,Wt,Sham
172,452412,11.587,Wt,Sham
173,452412,12.151,Wt,Sham


## Export results in a csv file
Get the means per animal and sort by surgery and genotype.


In [33]:
result = result.groupby('Mouse ID').mean().merge(infodata[['Mouse ID', 'Genotype', 'Surgery']], on= 'Mouse ID', how= 'left')
result = result.sort_values(by = ['Surgery', 'Genotype'])
display(result)
result.to_csv('Vessel density ALL.csv', index= False)

,Mouse ID,%Area image,Genotype,Surgery
1,445777,9.787750,CKO,Sham
2,445778,9.078000,CKO,Sham
4,445780,11.621750,CKO,Sham
13,452413,9.199714,CKO,Sham
21,468721,12.606375,CKO,Sham
12,452412,12.509625,Wt,Sham
14,452414,11.171625,Wt,Sham
15,452415,10.309125,Wt,Sham
19,468719,11.028500,Wt,Sham
20,468720,10.701500,Wt,Sham
